In [1]:
import os
from os.path import isfile
import nibabel as nb
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from scipy.stats import norm
import pickle
from nilearn import plotting
from scipy import ndimage
from kernel import kernel_conv
from compile_studies import compile_studies
from template import prior, affine, shape, pad_shape
from main_effect import *
from mytime import tic, toc

THRESH = 0.001

cwd = os.getcwd()
raw_folder = f'{cwd}/DataRaw/'
pickle_folder = f'{cwd}/DataPickle/'

'''filename = askopenfilename()
df = pd.read_excel(filename, engine='openpyxl', header=None)'''

meta_df = pd.read_excel(raw_folder + 'Metastocalculate.xlsx', engine='openpyxl', header=None)
with open(pickle_folder + 'experiments.pickle', 'rb') as f:
    exp_all, tasks = pickle.load(f)

row_idx = 3

exp_name1 = meta_df.iloc[row_idx, 1]
exp_idxs1 = compile_studies(meta_df, row_idx, tasks)
exp_df1 = exp_all.loc[exp_idxs1].reset_index(drop=True)
s1 = list(range(exp_df1.shape[0]))

exp_name2 = meta_df.iloc[row_idx + 1, 1]
exp_idxs2 = compile_studies(meta_df, row_idx+1, tasks)
exp_df2 = exp_all.loc[exp_idxs2].reset_index(drop=True)
s2 = list(range(exp_df2.shape[0]))

In [2]:
cwd = os.getcwd()
mask_folder = f"{cwd}/MaskenEtc/"
try:
    os.makedirs(f"{cwd}/ALE/rDiffs/Images")
    os.makedirs(f"{cwd}/ALE/rDiffs/Null")
    os.mkdirs(f"{cwd}/ALE/PMaps")    
except:
    pass

uc = 0.001
s0 = list(range(exp_df1.shape[0]))
# highest possible ale value if every study had a peak at the same location.
mb = 1
for i in s0:
    mb = mb*(1-np.max(exp_df1.at[i, 'Kernels']))

# define bins for histogram
bin_edges = np.arange(0.00005,1-mb+0.001,0.0001)
bin_centers = np.arange(0,1-mb+0.001,0.0001)

sample_n = 2500
target_n = 17

sample_space = np.array(np.where(prior == True))
peaks = np.array([exp_df1.XYZ[i].T[:,:3] for i in s0], dtype=object)

In [3]:
def create_samples(s0, sample_space, sample_n, target_n):
    samples = np.zeros((sample_n, target_n))
    s_perm = s0.copy()
    for i in range(sample_n):
        np.random.shuffle(s_perm)
        samples[i,:] = s_perm[:target_n]
    samples = np.unique(samples, axis=0)
    unique_samples = samples.shape[0]
    while unique_samples < sample_n:
        np.random.shuffle(s_perm)
        samples = np.vstack((samples, s_perm[:target_n]))
        samples = np.unique(samples, axis=0)
        unique_samples = samples.shape[0]
    return samples.astype(int)

In [4]:
# Cluster Null
max_ale, max_cluster, thresh = zip(*Parallel(n_jobs=8, verbose=10)(delayed(compute_noise_max)(s0 = s1,
                                                                                              sample_space = sample_space,
                                                                                              num_peaks = exp_df1.Peaks,
                                                                                              kernels = exp_df1.Kernels,
                                                                                              bin_centers=bin_centers,
                                                                                              bin_edges=bin_edges,
                                                                                              target_n=17) for i in range(500)))
cut_cluster = np.percentile(max_cluster, 95)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    3.2s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    4.2s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    4.4s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    5.8s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    6.7s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:    7.8s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:    9.1s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:   11.1s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:   12.6s
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:   14.4s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:   15.8s
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:   18.0s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:   19.8s
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:   21.8s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   23.8s
[Parallel(

In [13]:
samples = create_samples(s1, sample_space, sample_n, target_n)

In [14]:
# SubALE
ma = compute_ma(s0, peaks, exp_df1.Kernels)
hx = compute_hx(s0, ma, bin_edges)


p_map = np.zeros((sample_n,sample_space[0].size))
ale = compute_ale(ma)

In [24]:
def compute_pmap(sample, ma, hx, bin_centers, cut_cluster):
    ale_null = compute_ale_null(sample, hx, bin_centers)
    ale = compute_ale(ma[sample])
    z = compute_z(ale, ale_null)
    z, max_cluster = compute_cluster(z, THRESH, sample_space=None, cut_cluster=cut_cluster)
    z[z > 0] = 1
    return z

In [41]:
prob_act = Parallel(n_jobs=8, verbose=2)(delayed(compute_pmap)(samples[i], ma, hx, bin_centers, 87) for i in range(1000))

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    6.1s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:   16.6s
[Parallel(n_jobs=8)]: Done 349 tasks      | elapsed:   34.5s
[Parallel(n_jobs=8)]: Done 632 tasks      | elapsed:  1.0min
[Parallel(n_jobs=8)]: Done 1000 out of 1000 | elapsed:  1.6min finished


In [42]:
prob_act = np.mean(prob_act, axis=0)